### Redes con tensorflow
#### Por Francisco Serradilla

#### Tareas

* [x] Entrenar el perceptrón multicapa suministrado.
* [x] Ampliarlo para meter más capas, usar activación relu y evaluar con un conjunto de test.
* [x] Entrenar con algunos de los problemas suministrados.
* (Opcional) Probar otros optimizadores.
* [x] (Opcional) Añadir dropout. ¿Mejora la generalización con alguno de los problemas suministrados?

### Introducción a tensorflow

In [2]:
# Creación de grafos

print('Loading tensorflow...')
import tensorflow as tf
print('Loaded')

# Create a Constant op that produces a 1x2 matrix.  The op is
# added as a node to the default graph.
# The value returned by the constructor represents the output of the Constant op.
matrix1 = tf.constant([[3., 3.]])

# Create another Constant that produces a 2x1 matrix.
matrix2 = tf.constant([[2.],[2.]])

# Create a Matmul op that takes 'matrix1' and 'matrix2' as inputs.
# The returned value, 'product', represents the result of the matrix multiplication.
product = tf.matmul(matrix1, matrix2)

product

Loading tensorflow...
Loaded


<tf.Tensor 'MatMul_1:0' shape=(1, 1) dtype=float32>

In [3]:
# Ejecutar un grafo

# Launch the default graph.
sess = tf.Session()
# To run the matmul op we call the session 'run()' method, passing ‘product' which represents the output of the matmul op.  This indicates to the call that we want to get the output of the matmul op back.
# All inputs needed by the op are run automatically by the session.  They typically are run in parallel.
# The call 'run(product)' thus causes the execution of three ops in the graph: the two constants and matmul. The output of the op is returned in 'result' as a numpy `ndarray` object.
result = sess.run(product)
print(result)
# ==> [[ 12.]]

# Close the Session when we're done.
sess.close()

[[12.]]


In [4]:
# Variables

# Create a Variable, that will be initialized to the scalar value 0.
state = tf.Variable(0, name="counter")

# Create an Op to add one to `state`.
one = tf.constant(1)
new_value = tf.add(state, one)
update = tf.assign(state, new_value)

# Variables must be initialized by running an `init` Op after having launched the graph.  
init_op = tf.global_variables_initializer() # We first have to add the `init` Op to the graph

with tf.Session() as sess: # Launch the graph and run the ops
    sess.run(init_op) # Run the 'init' op
    print(sess.run(state)) # Print the initial value of 'state'
    for _ in range(3): # Run the op that updates 'state' and print 'state'.
        sess.run(update)
        print(sess.run(state))

Instructions for updating:
Colocations handled automatically by placer.
0
1
2
3


In [5]:
# fetches

input1 = tf.constant(3.0)
input2 = tf.constant(2.0)
input3 = tf.constant(5.0)
intermed = tf.add(input2, input3)
mul = tf.multiply(input1, intermed)

with tf.Session() as sess:
    result = sess.run([mul, intermed]) # fetches -> lo que se quiere obtener del cálculo
    print(result)

[21.0, 7.0]


In [6]:
# placeholders

input1 = tf.placeholder(tf.float32)
input2 = tf.placeholder(tf.float32)
output = tf.multiply(input1, input2)

with tf.Session() as sess:
    print(sess.run(output, feed_dict={input1:[7.], input2:[2.]}))

[14.]


In [7]:
# Carga ejemplos

import numpy as np

ni = 2 # número de entradas
nh = 10 # número de neuronas en capa oculta
no = 3 # número de neuronas en capa de salida
capas = 5 #número de capas ocultas de la red
datos_cargados = "circulo"

# carga datos de entrenamiento
d = np.loadtxt("samples/circulo.txt")
inputs = d[:,:ni]

outputs = d[:,ni:]


In [8]:
# Creación de la red
# Define un perceptrón multicapa con 2 capas usando RMS como medida del error y back proagation como algoritmo de entrenamiento

import tensorflow as tf

# un placeholder es un punto de comunicación de datos entre nuestra app y la librería tensorflow
# creamos un placeholder para indicar las entradas a la red de todos los ejemplos
e = tf.placeholder(tf.float32, [None, ni]) # None indica que la primera dimensión no es fija (el número de ejemplos)

# un nuevo placeholder para indicar las salidas deseadas
d = tf.placeholder(tf.float32, [None, no])

# las variables son matrices que residen fuera de python y no son modificadas directamente por nuestra app
W1 = tf.Variable(tf.random_uniform([ni, nh], -0.1, 0.1, dtype=tf.float32))
b1 = tf.Variable(tf.random_uniform([nh], -0.1, 0.1, dtype=tf.float32))
W2 = tf.Variable(tf.random_uniform([nh, no], -0.1, 0.1, dtype=tf.float32))
b2 = tf.Variable(tf.random_uniform([no], -0.1, 0.1, dtype=tf.float32))

# aquí definimos la propagación de la red
s1 = tf.nn.sigmoid(tf.add(tf.matmul(e, W1), b1))
s = tf.nn.sigmoid(tf.add(tf.matmul(s1, W2), b2))

print("\ndatos cargados:", datos_cargados,"\n")
print("s1")
print(s1)


datos cargados: circulo 

s1
Tensor("Sigmoid:0", shape=(?, 10), dtype=float32)


### Entrenar el perceptrón multicapa suministrado

In [9]:
# Entrenamiento

# definimos la función a minimizar (error cuadrático medio)
vRMS = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(d,s)), reduction_indices=0))
RMS = tf.reduce_mean(vRMS)

# definimos el algoritmo de aprendizaje a utilizar (descenso del gradiente sobre la función de coste)
train_step = tf.train.RMSPropOptimizer(0.01).minimize(RMS)

# definimos qué es un acierto  
correct_prediction = tf.equal(tf.argmax(s,1), tf.argmax(d,1))

# definimos la precisión como el porcentaje de aciertos
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# definimos la inicialización de las variables internas
init = tf.global_variables_initializer()

# iniciamos sesión con tensorflow y ejecutamos la inicialización
sess = tf.Session()
sess.run(init)

print("\nEntrenamiento para datos",datos_cargados, "\n")

# ejecutamos el número de epochs indicados
epochs = 1000
trace = 100
for i in range(1,epochs+1):
  sess.run(train_step, feed_dict={e: inputs, d: outputs})
  if i%trace == 0:
    print ('Epoch: %d' % i)
    # calculamos RMS y precisión y la escribimos por pantalla
    print('RMS:',sess.run(RMS, feed_dict={e: inputs, d: outputs}))
    print ('Accuracy:',sess.run(accuracy, feed_dict={e: inputs, d: outputs}))
    
sess.close()

Instructions for updating:
Use tf.cast instead.

Entrenamiento para datos circulo 

Epoch: 100
RMS: 0.4554516
Accuracy: 0.48
Epoch: 200
RMS: 0.45537496
Accuracy: 0.48
Epoch: 300
RMS: 0.42150363
Accuracy: 0.56
Epoch: 400
RMS: 0.34897864
Accuracy: 0.84
Epoch: 500
RMS: 0.2962158
Accuracy: 0.84
Epoch: 600
RMS: 0.25511953
Accuracy: 0.96
Epoch: 700
RMS: 0.22278762
Accuracy: 1.0
Epoch: 800
RMS: 0.19427179
Accuracy: 1.0
Epoch: 900
RMS: 0.16483466
Accuracy: 1.0
Epoch: 1000
RMS: 0.13522689
Accuracy: 1.0


### Entrenar con algunos de los problemas suministrados.
Para entrenar con un problema, ejecutar la carga de datos de ese problema y posteriormente ejecutar el entrenamiento 

In [ ]:
# Cargar datos morosos 

ni = 9 # número de entradas
nh = 3 # número de neuronas en capa oculta
no = 1 # número de neuronas en capa de salida
capas = 2 #número de capas ocultas de la red
datos_cargados = "morosos"

# carga datos de entrenamiento
d = np.loadtxt("samples/morosos-ent.txt")
inputs = d[:,:ni]

outputs = d[:,ni:]

t = np.loadtxt("samples/morosos-tst.txt")
inputsT = t[:,:ni]
outputsT = t[:,ni:]

In [ ]:
# Cargar data2 class

ni = 2 # número de entradas
nh = 7 # número de neuronas en capa oculta
no = 1 # número de neuronas en capa de salida
capas = 2 #número de capas ocultas de la red
datos_cargados = "data 3classes nonlinear"

# carga datos de entrenamiento
d = np.loadtxt("samples/data_3classes_nonlinear_2D.txt")
inputs = d[:,:ni]

outputs = d[:,ni:]

## Para este caso no hay conjunto de test 
t = np.loadtxt("samples/data_3classes_nonlinear_2D.txt")
inputsT = t[:,:ni]
outputsT = t[:,ni:]


In [17]:
# Cargar datos quinielas

ni = 60 # número de entradas
nh = 60 # número de neuronas en capa oculta
no = 3 # número de neuronas en capa de salida
capas = 25 #número de capas ocultas de la red
datos_cargados = "quinielas"

# carga datos de entrenamiento
d = np.loadtxt("samples/quinielas60-3-trn.txt")
inputs = d[:,:ni]
outputs = d[:,ni:]

t = np.loadtxt("samples/quinielas60-3-tst.txt")
inputsT = t[:,:ni]
outputsT = t[:,ni:]

In [ ]:
# Cargar datos aprobados 

ni = 3 # número de entradas
nh = 7 # número de neuronas en capa oculta
no = 1 # número de neuronas en capa de salida
capas = 10 #número de capas ocultas de la red
datos_cargados = "aprobado"

# carga datos de entrenamiento
d = np.loadtxt("samples/aprobado-ent.txt")
inputs = d[:,:ni]
outputs = d[:,ni:]

t = np.loadtxt("samples/aprobado-tst.txt")
inputsT = t[:,:ni]
outputsT = t[:,ni:]



### Ampliar el perceptrón multicapa suministrado para meter más capas, usar activación relu y evaluar con un conjunto de test.



In [19]:
# Creación de la red
# Define un perceptrón multicapa con 2 capas usando RMS como medida del error y back proagation como algoritmo de entrenamiento

import tensorflow as tf

# un placeholder es un punto de comunicación de datos entre nuestra app y la librería tensorflow
# creamos un placeholder para indicar las entradas a la red de todos los ejemplos
e = tf.placeholder(tf.float32, [None, ni]) # None indica que la primera dimensión no es fija (el número de ejemplos)

# un nuevo placeholder para indicar las salidas deseadas
d = tf.placeholder(tf.float32, [None, no])


# las variables son matrices que residen fuera de python y no son modificadas directamente por nuestra app
W = []
b = []

W.append(tf.Variable(tf.glorot_uniform_initializer()((ni, nh),dtype=tf.float32)))
b.append(tf.Variable(tf.glorot_uniform_initializer()([nh], dtype=tf.float32)))
for _ in range (1,capas-1):
    W.append(tf.Variable(tf.glorot_uniform_initializer()((nh, nh), dtype=tf.float32)))
    b.append(tf.Variable(tf.glorot_uniform_initializer()([nh], dtype=tf.float32)))

W.append(tf.Variable(tf.glorot_uniform_initializer()((nh, no),  dtype=tf.float32)))
b.append(tf.Variable(tf.glorot_uniform_initializer()([no], dtype=tf.float32)))

# aquí definimos la propagación de la red
s = []
s.append(tf.nn.relu(tf.add(tf.matmul(e, W[0]), b[0])))
for i in range (capas - 1):
    s.append(tf.nn.relu(tf.add(tf.matmul(s[i], W[i+1]), b[i+1])))

print("\ndatos cargados:", datos_cargados,"\n")




datos cargados: quinielas 



In [24]:
# Entrenamiento

# definimos la función a minimizar (error cuadrático medio)
vRMS = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(d,s[capas-1])), reduction_indices=0))
RMS = tf.reduce_mean(vRMS)

# definimos el algoritmo de aprendizaje a utilizar (descenso del gradiente sobre la función de coste)
train_step = tf.train.AdamOptimizer().minimize(RMS)

# definimos qué es un acierto  
correct_prediction = tf.equal(tf.argmax(s[capas-1],1), tf.argmax(d,1))

# definimos la precisión como el porcentaje de aciertos
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# definimos la inicialización de las variables internas
init = tf.global_variables_initializer()

# iniciamos sesión con tensorflow y ejecutamos la inicialización
sess = tf.Session()
sess.run(init)

print("\nEntrenamiento para datos",datos_cargados, "\n")

# ejecutamos el número de epochs indicados
epochs = 1000
trace = 100
for i in range(1,epochs+1):
  sess.run(train_step, feed_dict={e: inputs, d: outputs})
  if i%trace == 0:
    print ('\nEpoch: %d' % i)
    # calculamos RMS y precisión y la escribimos por pantalla
    print('RMS:',sess.run(RMS, feed_dict={e: inputs, d: outputs}))
    print('RMS test:',sess.run(RMS, feed_dict={e: inputsT, d: outputsT}))
    print ('Accuracy:',sess.run(accuracy, feed_dict={e: inputs, d: outputs}))
    print ('Accuracy test:',sess.run(accuracy, feed_dict={e: inputsT, d: outputsT}))
    
sess.close()


Entrenamiento para datos quinielas 


Epoch: 100
RMS: 0.42289054
RMS test: 0.43324438
Accuracy: 0.58518517
Accuracy test: 0.5272727

Epoch: 200
RMS: 0.34734538
RMS test: 0.5013001
Accuracy: 0.7225589
Accuracy test: 0.4121212

Epoch: 300
RMS: 0.26921713
RMS test: 0.5202158
Accuracy: 0.86599326
Accuracy test: 0.4121212

Epoch: 400
RMS: 0.17903815
RMS test: 0.57969797
Accuracy: 0.93468016
Accuracy test: 0.4181818

Epoch: 500
RMS: 0.25668693
RMS test: 0.5694006
Accuracy: 0.8922559
Accuracy test: 0.38787878

Epoch: 600
RMS: 0.10844108
RMS test: 0.57723135
Accuracy: 0.9878788
Accuracy test: 0.42424244

Epoch: 700
RMS: 0.14564045
RMS test: 0.57716125
Accuracy: 0.95555556
Accuracy test: 0.44242424

Epoch: 800
RMS: 0.11154207
RMS test: 0.5837092
Accuracy: 0.96498317
Accuracy test: 0.45454547

Epoch: 900
RMS: 0.11098633
RMS test: 0.5880737
Accuracy: 0.96498317
Accuracy test: 0.44848484

Epoch: 1000
RMS: 0.118270814
RMS test: 0.58203673
Accuracy: 0.96363634
Accuracy test: 0.44242424


### Probar otros optimizadores

Se utilizaron los optimizadores:
- **RMSPropOptimizer**
RMSprop es un optimizado rápido y popular diseñado para redes de neuronas que utiliza métodos de cambio de tasa para el aprendizaje adaptativo. Este algoritmo trata de resolver el problema de que los gradientes pueden variar ampliamente. Hace una generalización que garantiza que todas las actualizaciones de los pesos sean del mismo tamaño. 

- **AdamOptimizer**
El algoritmo de optimización de Adam es una extensión del descenso de gradiente que se utiliza para deep Learning y normalmente para visión por computadora y procesamiento de lenguaje natural. Lo que hace es actualizar los pesos iterativamente basándose en los datos entrenados, a diferencia del algoritmo tradicional que mantiene una tasa de aprendizaje a lo largo de todo el entrenamiento. 
Adam ayuda a obtener buenos resultados de forma rápida y se recomienda que sea el algoritmo por defecto. 


Referencias: 
- Brownlee, J. (2017). Gentle Introduction to the Adam Optimization Algorithm for Deep Learning. Machine Learning Mastery. Retrieved 1 May 2019, from https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/
- Understanding RMSprop — faster neural network learning. (2018). Towards Data Science. Retrieved 1 May 2019, from https://towardsdatascience.com/understanding-rmsprop-faster-neural-network-learning-62e116fcf29a



###  Añadir dropout.

In [26]:
# Creación de la red DROPOUT

import tensorflow as tf

# un placeholder es un punto de comunicación de datos entre nuestra app y la librería tensorflow
# creamos un placeholder para indicar las entradas a la red de todos los ejemplos
e = tf.placeholder(tf.float32, [None, ni]) # None indica que la primera dimensión no es fija (el número de ejemplos)

# un nuevo placeholder para indicar las salidas deseadas
d = tf.placeholder(tf.float32, [None, no])

# las variables son matrices que residen fuera de python y no son modificadas directamente por nuestra app
W = []
b = []

W.append(tf.Variable(tf.glorot_uniform_initializer()((ni, nh),dtype=tf.float32)))
b.append(tf.Variable(tf.glorot_uniform_initializer()([nh], dtype=tf.float32)))
for _ in range (1,capas-1):
    W.append(tf.Variable(tf.glorot_uniform_initializer()((nh, nh), dtype=tf.float32)))
    b.append(tf.Variable(tf.glorot_uniform_initializer()([nh], dtype=tf.float32)))

W.append(tf.Variable(tf.glorot_uniform_initializer()((nh, no),  dtype=tf.float32)))
b.append(tf.Variable(tf.glorot_uniform_initializer()([no], dtype=tf.float32)))

# aquí definimos la propagación de la red con DROPOUT
s = []
s.append(tf.nn.relu(tf.add(tf.matmul(e, W[0]), b[0])))

y = tf.placeholder("float", [None, 3])
keep_prob = tf.placeholder(tf.float32)  
drop_out = tf.nn.dropout(s[0], keep_prob) 

s.append(tf.matmul(s[0], W[1]) + b[1]) #DROPOUT primera capa oculta

for i in range (1, capas - 1):
    s.append(tf.nn.relu(tf.add(tf.matmul(s[i], W[i+1]), b[i+1])))# Entrenamiento CON DROPOUT 


In [27]:
#Entramiento de la red con DROPOUT

vRMS = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(d,s[capas-1])), reduction_indices=0))
RMS = tf.reduce_mean(vRMS)

# definimos el algoritmo de aprendizaje a utilizar (descenso del gradiente sobre la función de coste)
train_step = tf.train.AdamOptimizer().minimize(RMS)

# definimos qué es un acierto  
correct_prediction = tf.equal(tf.argmax(s[capas-1],1), tf.argmax(d,1))

# definimos la precisión como el porcentaje de aciertos
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# definimos la inicialización de las variables internas
init = tf.global_variables_initializer()

# iniciamos sesión con tensorflow y ejecutamos la inicialización
sess = tf.Session()
sess.run(init)

# ejecutamos el número de epochs indicados
epochs = 1000
trace = 100
for i in range(1,epochs+1):
  sess.run(train_step, feed_dict={e: inputs, d: outputs})
  if i%trace == 0:
    print ('Epoch: %d' % i)
    # calculamos RMS y precisión y la escribimos por pantalla
    print('RMS:',sess.run(RMS, feed_dict={e: inputs, d: outputs}))
    print('RMS test:',sess.run(RMS, feed_dict={e: inputsT, d: outputsT}))
    print ('Accuracy:',sess.run(accuracy, feed_dict={e: inputs, d: outputs}))
    print ('Accuracy test:',sess.run(accuracy, feed_dict={e: inputsT, d: outputsT}))
    
sess.close()

Epoch: 100
RMS: 0.5027157
RMS test: 0.5170534
Accuracy: 0.34074074
Accuracy test: 0.28484848
Epoch: 200
RMS: 0.4668431
RMS test: 0.5336231
Accuracy: 0.41144782
Accuracy test: 0.29090908
Epoch: 300
RMS: 0.4692955
RMS test: 0.5534565
Accuracy: 0.42087543
Accuracy test: 0.2969697
Epoch: 400
RMS: 0.42442146
RMS test: 0.5557351
Accuracy: 0.68686867
Accuracy test: 0.44242424
Epoch: 500
RMS: 0.35204205
RMS test: 0.5870443
Accuracy: 0.84175086
Accuracy test: 0.44848484
Epoch: 600
RMS: 0.32112747
RMS test: 0.60179
Accuracy: 0.96430975
Accuracy test: 0.46060607
Epoch: 700
RMS: 0.31893826
RMS test: 0.6018736
Accuracy: 0.9683502
Accuracy test: 0.45454547
Epoch: 800
RMS: 0.31891736
RMS test: 0.59941095
Accuracy: 0.96700335
Accuracy test: 0.45454547
Epoch: 900
RMS: 0.3181843
RMS test: 0.5931241
Accuracy: 0.9690236
Accuracy test: 0.47272727
Epoch: 1000
RMS: 0.31816477
RMS test: 0.590388
Accuracy: 0.9690236
Accuracy test: 0.4909091


### Explicación DROPOUT (datos quinielas)

El ejemplo mas complejo que hemos cargado en esta prática es el de "quinielas" y por ello vamos a razonar el dropout con estos datos para que sea mas visual.En el resto de ejemplos también se produciría.
Al realizar el entrenamiento con dropout y sin dropout (mostrados arriba) se puede observar que con dropout se generaliza un poco mas la red y, por lo tanto, en la precisión (accuracy) de los resultados en test son algo mejores en la ejecución con dropout.

Ejemplo quinielas:

Sin dropout el entramiento nos ha llegado a salir una precisión muy buena (en torno al 0.95) y, sin embargo, en el test dan resultados bastante malos, aproximidamente 0.44. Si se introduce dropout, en el entramiento podemos alcanzar una probabilidad similar, pero en el test si se consiguen resultados mejores (0.5 aprox.).

Por ello, como respuesta a la pregunta del enunciado, se puede decir que si es posible mejorar la generalización con dropout.
 